In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import itertools
import subprocess

def show(state):
    plt.imshow(state)
    
def step(state):
    nxt = np.zeros_like(state)
    
    ny, nx = state.shape
    
    for y in range(ny):
        for x in range(nx):
            cnt = 0
            for yy in range(max(y - 1, 0), min(y + 2, ny)):
                for xx in range(max(x - 1, 0), min(x + 2, nx)):
                    if state[yy][xx]:
                        cnt = cnt + 1

            v = 1
            if state[y][x]:
                cnt = cnt - 1
                if cnt <= 1 or cnt >= 4:
                    v = 0
            else:
                if cnt != 3:
                    v = 0
            nxt[y][x] = v
    return nxt

In [ ]:
def glucose(cnf, verbose = False):
    m = 0
    for clause in cnf:
        for lit in clause:
            if lit > m:
                m = lit
    
    tempcnf = "temp.cnf"
    with open(tempcnf, "w") as f:
        print("p cnf", m, len(cnf), file=f)
        for clause in cnf:
            f.write(" ".join(map(str, clause)))
            f.write(" 0\n")
            
    ret = subprocess.run(["./glucose/simp/glucose", "-model", tempcnf], stdout=subprocess.PIPE)
    out = ret.stdout.decode('utf-8')
    
    sln = None
    model = []
    for l in out.splitlines():
        if l.startswith('s'):
            sln = l[2:]
        if l.startswith('v'):
            model = list(map(int, l.split(' ')[1:-1]))
            
    if verbose:
        print(out)
            
    return sln, model

In [ ]:
def backLifeCnf1(state):
    ny, nx = state.shape
    nxy = nx * ny

    def ix(x, y, t):
        return t * nxy + y * nx + x + 1
    
    neighbors = [\
        (-1, 1), (0, 1), (1, 1), \
        (-1, 0),         (1, 0), \
        (-1,-1), (0,-1), (1,-1), \
    ]
    
    cnf = []
    
    for y in range(1, ny - 1):
        for x in range(1, nx - 1):
            # (8 choose 4) like (-x' V -xa V -xb V -xc V -xd)
            for neigh in itertools.combinations(neighbors, 4):
                clause = [-ix(x, y, 1)]
                for off in neigh:
                    ox, oy = off
                    clause.append(-ix(x + ox, y + oy, 0))
                cnf.append(clause)
            # (8 choose 7) like (-x' V xa V ... V xg)
            for neigh in itertools.combinations(neighbors, 7):
                clause = [-ix(x, y, 1)]
                for off in neigh:
                    ox, oy = off
                    clause.append(ix(x + ox, y + oy, 0))
                cnf.append(clause)
            # (8 choose 6) like (-x' V x V xa V ... V xf)
            for neigh in itertools.combinations(neighbors, 6):
                clause = [-ix(x, y, 1), ix(x, y, 0)]
                for off in neigh:
                    ox, oy = off
                    clause.append(ix(x + ox, y + oy, 0))
                cnf.append(clause)
            # (8 choose 3) like (x' V -xa V -xb V -xc V xd V ... xh) complementing just 3
            for three in itertools.combinations(range(8), 3):
                clause = [ix(x, y, 1)]
                for off in neighbors:
                    ox, oy = off
                    clause.append(ix(x + ox, y + oy, 0))
                for t in three:
                    clause[t + 1] = -clause[t + 1]
                cnf.append(clause)
            # (8 choose 2) like (x' V -x V -xa V -xb V xc V ... xg) complementing just 2, omitting 1
            for two in itertools.combinations(range(8), 2):
                clause = [ix(x, y, 1)]
                for off in neighbors:
                    ox, oy = off
                    clause.append(ix(x + ox, y + oy, 0))
                for t in two:
                    clause[t + 1] = - clause[t + 1]
                for i in range(1, len(clause)):
                    if clause[i] > 0:
                        del clause[i]
                        break
                cnf.append(clause)
    
    lx = nx - 1
    ly = ny - 1
    corners = [(ix(0, 0, 1), ix(0, 0, 0), ix(0, 1, 0), ix(1, 1, 0), ix(1, 0, 0)),
               (ix(lx, 0, 1), ix(lx, 0, 0), ix(lx, 1, 0), ix(lx-1, 1, 0), ix(lx-1, 0, 0)),
               (ix(0, ly, 1), ix(0, ly, 0), ix(0, ly-1, 0), ix(1, ly-1, 0), ix(1, ly, 0)),
               (ix(lx, ly, 1), ix(lx, ly, 0), ix(lx, ly-1, 0), ix(lx-1, ly-1, 0), ix(lx-1, ly, 0))]
    for corner in corners:
        xp, x, a, b, c = corner
        cnf.extend([
            # if we were dead OR alive
                # and zero are alive
                    [-xp, a,  b,  c], #die
                # and one is alive
                    [-xp,  a,  b, -c],
                    [-xp,  a, -b,  c],
                    [-xp, -a,  b,  c], #die
                # and three are alive
                    [ xp, -a, -b, -c], #live
            
            # if we were dead
                # and two are alive
                    [-xp,  x,  a, -b, -c],
                    [-xp,  x, -a,  b, -c],
                    [-xp,  x, -a, -b,  c], #die
            
            # if we were alive
                # and two are alive
                    [ xp, -x,  a, -b, -c],
                    [ xp, -x, -a,  b, -c],
                    [ xp, -x, -a, -b,  c], #live
        ])
    
    
    def edge_neighbors(x, y, t):
        ln = []
        for off in neighbors:
            ox, oy = off
            tx = x + ox
            ty = y + oy
            if tx >= 0 and tx < nx and ty >= 0 and ty < ny:
                ln.append(ix(tx, ty, t))
        return ln
    def edge_clauses(xp, x, lits):
        a, b, c, d, e = lits
        
        return [
        # If we are alive or dead
            # and zero are alive
                [-xp,  a,  b,  c,  d,  e], # dead
            # and one is alive
                [-xp,  a,  b,  c,  d, -e],
                [-xp,  a,  b,  c, -d,  e],
                [-xp,  a,  b, -c,  d,  e],
                [-xp,  a, -b,  c,  d,  e],
                [-xp, -a,  b,  c,  d,  e], # dead
            # and four are alive
                [-xp,  a, -b, -c, -d, -e],
                [-xp, -a,  b, -c, -d, -e],
                [-xp, -a, -b,  c, -d, -e],
                [-xp, -a, -b, -c,  d, -e],
                [-xp, -a, -b, -c, -d,  e], # dead
            # and five are alive
                [-xp, -a, -b, -c, -d, -e], # dead
            # and three are alive
                [ xp,  a,  b, -c, -d, -e],
                [ xp,  a, -b,  c, -d, -e],
                [ xp,  a, -b, -c,  d, -e],
                [ xp,  a, -b, -c, -d,  e],
                [ xp, -a,  b,  c, -d, -e],
                [ xp, -a,  b, -c,  d, -e],
                [ xp, -a,  b, -c, -d,  e],
                [ xp, -a, -b,  c,  d, -e],
                [ xp, -a, -b,  c, -d,  e],
                [ xp, -a, -b, -c,  d,  e], # alive

        # If we are dead
            # and two are alive
                [-xp,  x,  a,  b,  c, -d, -e],
                [-xp,  x,  a,  b, -c, -d,  e],
                [-xp,  x,  a,  b, -c,  d, -e],
                [-xp,  x,  a, -b, -c,  d,  e],
                [-xp,  x,  a, -b,  c,  d, -e],
                [-xp,  x,  a, -b,  c, -d,  e],
                [-xp,  x, -a,  b,  c,  d, -e],
                [-xp,  x, -a,  b,  c, -d,  e],
                [-xp,  x, -a,  b, -c,  d,  e],
                [-xp,  x, -a, -b,  c,  d,  e], # dead
        # If we are alive
            # and two are alive
                [ xp, -x,  a,  b,  c, -d, -e],
                [ xp, -x,  a,  b, -c, -d,  e],
                [ xp, -x,  a,  b, -c,  d, -e],
                [ xp, -x,  a, -b, -c,  d,  e],
                [ xp, -x,  a, -b,  c,  d, -e],
                [ xp, -x,  a, -b,  c, -d,  e],
                [ xp, -x, -a,  b,  c,  d, -e],
                [ xp, -x, -a,  b,  c, -d,  e],
                [ xp, -x, -a,  b, -c,  d,  e],
                [ xp, -x, -a, -b,  c,  d,  e], # alive
        ]
    
    for x in range(1, lx):
        cnf.extend(edge_clauses(ix(x, 0, 1), ix(x, 0, 0), edge_neighbors(x,  0, 0)))
        cnf.extend(edge_clauses(ix(x, ly, 1), ix(x, ly, 0), edge_neighbors(x, ly, 0)))
    for y in range(1, ly):
        cnf.extend(edge_clauses(ix(0, y, 1), ix(0, y, 0), edge_neighbors( 0, y, 0)))
        cnf.extend(edge_clauses(ix(lx, y, 1), ix(lx, y, 0), edge_neighbors(lx, y, 0)))
        
    for y in range(ny):
        for x in range(nx):
            if state[y][x] == 1:
                cnf.append([ix(x, y, 1)])
            elif state[y][x] == 0:
                cnf.append([-ix(x, y, 1)])

    return cnf

target = np.array([ 
    [0, 0, 0, 0, 0], 
    [0, 1, 1, 1, 0], 
    [0, 0, 0, 1, 0], 
    [0, 0, 1, 0, 0], 
    [0, 0, 0, 0, 0], 
])
   
cnf = backLifeCnf1(target)
#for clause in cnf:
#    print (clause[0],(sorted(clause[1:])))
res = glucose(cnf)
if res[0] == 'SATISFIABLE':
    arr = np.zeros(2 * target.shape[0] * target.shape[1], dtype=bool)
    for vs in res[1]:
        v = int(vs)
        arr[abs(v) - 1] = v > 0
        
    arr = arr.reshape((2, target.shape[0], target.shape[1]))
    f, ax = plt.subplots(2)
    ax[0].imshow(arr[0])
    ax[1].imshow(arr[1])
else:
    print("UNSATISFIABILE")
